In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

In [ ]:
# data paths
mixture_directory = "data/mixture"
vocals_directory = "data/vocals"

In [ ]:
# parameters to play around with to better optimize training
patch_size = 128
stride = 64
batch_size = 16
epochs = 20
model_save_path = "vocal_isolator.pth"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # will utilize GPU if possible to train/test

In [ ]:
class SpectrogramDataset(Dataset):
    def __init__(self):
    
    def __len__(self):
        
    def __getitem__(self, index):

In [ ]:
class UNet(nn.Module):  #TODO: Should we be using a CNN or a UNET?
    def __init__(self):
        
    def forward(self, x):

In [ ]:
# training function to be called
def train(model, dataloader, criterion, optimizer, num_epochs, checkpoint_path="checkpoints"):
    os.makedirs(checkpoint_path, exist_ok=True)
    model.train()
    checkpoint_interval = 5

    for epoch in range(1, num_epochs + 1):
        running_loss = 0.0
        num_batches = 0

        for spectrograms, labels in dataloader:
            # Move to GPU
            spectrograms, labels = spectrograms.to(device), labels.to(device)

            # Forward pass
            outputs = model(spectrograms)
            loss = criterion(outputs, labels)

            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            num_batches += 1

        # Calculate average loss
        epoch_loss = running_loss / num_batches

        print(f"Epoch {epoch}/{num_epochs} — Average Loss: {epoch_loss:.4f}")

        # Periodic checkpoints
        if epoch % checkpoint_interval == 0:
            ckpt = {
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': epoch_loss,
            }
            torch.save(ckpt, os.path.join(checkpoint_path, f"sep_epoch_{epoch}.pth"))

    return model